## 텍스트 전처리 실습

다음뉴스 본문을 크롤링하고, 본문 텍스트 처리(Text clearing, Tokenizing, Vectorization) 실습

In [1]:
import requests
import urllib3
import json

from bs4 import BeautifulSoup

In [2]:
def get_html(url, encoding='UTF-8') :
    r = requests.get(url, headers = None, timeout = 3.0)
    r.encoding = encoding
    return r.text

def get_soup(url, encoding='UTF-8') :
    return BeautifulSoup(get_html(url, encoding), "html")

In [3]:
#t1 -> https://news.v.daum.net/v/20210902132102417
#t2 -> https://news.v.daum.net/v/20210902060009685
s = get_soup("https://news.v.daum.net/v/20210902132102417")
test_doc = s.find("div", {"class" : "news_view"}).text.strip()

In [5]:
print(test_doc)

김한정 의원 "4년만에 88.2% 증가..감독당국 선제적 대응해야" 





            자료=금융감독원
           


            [쿠키뉴스] 유수환 기자 = 2일 국회 정무위원회 소속 더불어민주당 김한정 의원은 부동산 PF대출이 빠른 속도로 증가하고 있어 금융 불안을 초래할 위험이 커지고 있다며 감독당국에 보다 적극적인 위험 관리대책을 주문했다.
          
금융감독원에 따르면 지난해 말 국내 금융권의 부동산 PF대출 잔액은 88조4,838억원으로 2016년말 (47조256억원)에 대비 4년만에 41조4582억원(88.2%) 증가했다. 부동산 PF대출 급증은 부동산경기 호조가 주된 배경이지만 코로나 영향으로 해외 대체투자가 어려운점(보험사), 부동산 직접투자 제한 영향(여전사, 보험사) 등도 증가 요인이다.
금융권역별로는 보험사가 36조3826억원으로 가장 많고 은행이 23조8572억원, 저축은행 6조 8647억원, 증권사 4조2691억원, 상호금융 3조3105억원 순이다.
부동산 PF대출 가운데 고정금리 비중은 79.2%였는데, 저축은행이 97.6%로 가장 높았고, 보험사 79.9%, 여전사 72.3%, 상호금융 61.0% 순이었다. 부동산 PF대출의 금리는 보험사가 평균 3.24%로 가장 낮았고, 상호금융 3.47%, 여전사 4.59%, 증권사 5.78%, 저축은행 6.91% 순이었다.
다만 아직 리스크에 노출되지는 않았다. 부동산 PF대출의 연체율은 전업권이 0.49%로 2016년 2.4%에 비해 대폭 하락했다. 고정이하 여신 잔액도 6,472억원으로 2016년말 1조 9,595억원 대비 큰 폭으로 감소했다.
김한정 의원은 “부동산시장의 활황세를 등에 업고 부동산 PF대출이 급속히 증가하고 있다. 부동산 PF대출의 연체율이나 부실여신 잔액이 줄어드는 등 외관상으로는 문제가 없는 것으로 보이지만 부동산 PF대출의 잠재적 위험을 사전에 예방하는 것이 중요하다”고 강조했다.
이어 “부동산 경기활황시 PF대출을 늘렸다 부

## 1. Text Clearing

정규표현식을 사용해서 텍스트에 필요없는 부분 제거

html에서 텍스트를 추출해서 문장 단위로 쪼개는 과정

In [19]:
import re, os, math
import collections

from collections import Counter

doublespace_pattern = re.compile(u"\s+", re.UNICODE)
bracket_pattern = re.compile(r"\([^()]*?\)|\[.*?\]|\<.*?\>|\{.*?\}|\【.*?】|\(.*?\(.*?\).*?\)")
junk_filter = re.compile(u"^[a-zA-Z0-9?><;,.{}()[\]\-_+=!@#$%\^&*|/\n\t\s']{8,}", re.UNICODE)
head_filter = re.compile(u"[ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\?\!\s]+= ", re.UNICODE)
text_filter = re.compile(u'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\?\!-%]', re.UNICODE)
delete_filter = re.compile(r"[\'\"″“”‘’\?\!,…▲△□■※☆★▷▶▽▼『』#○●☆★＃◇◆ㅁ]")
space_filter = re.compile(r"[*·…‥♡♥\:=]|\.{2}")

In [20]:
def only_text(text):
    return doublespace_pattern.sub(' ',text_filter.sub(' ', text)).strip()

def text_clearing(text):
    return doublespace_pattern.sub(' ', space_filter.sub(' ', delete_filter.sub('', text))).strip()

def clear_bracket_regex(text):
    return bracket_pattern.sub("", text)
        
def xplit(value, delimiters = []):
    if not delimiters :
        return value.split()
    return re.split('|'.join([re.escape(delimiter) for delimiter in delimiters]), value)

In [8]:
only_text(test_doc)

'김한정 의원 "4년만에 88.2% 증가..감독당국 선제적 대응해야" 자료 금융감독원 쿠키뉴스 유수환 기자 2일 국회 정무위원회 소속 더불어민주당 김한정 의원은 부동산 PF대출이 빠른 속도로 증가하고 있어 금융 불안을 초래할 위험이 커지고 있다며 감독당국에 보다 적극적인 위험 관리대책을 주문했다. 금융감독원에 따르면 지난해 말 국내 금융권의 부동산 PF대출 잔액은 88조4 838억원으로 2016년말 47조256억원 에 대비 4년만에 41조4582억원 88.2% 증가했다. 부동산 PF대출 급증은 부동산경기 호조가 주된 배경이지만 코로나 영향으로 해외 대체투자가 어려운점 보험사 부동산 직접투자 제한 영향 여전사 보험사 등도 증가 요인이다. 금융권역별로는 보험사가 36조3826억원으로 가장 많고 은행이 23조8572억원 저축은행 6조 8647억원 증권사 4조2691억원 상호금융 3조3105억원 순이다. 부동산 PF대출 가운데 고정금리 비중은 79.2%였는데 저축은행이 97.6%로 가장 높았고 보험사 79.9% 여전사 72.3% 상호금융 61.0% 순이었다. 부동산 PF대출의 금리는 보험사가 평균 3.24%로 가장 낮았고 상호금융 3.47% 여전사 4.59% 증권사 5.78% 저축은행 6.91% 순이었다. 다만 아직 리스크에 노출되지는 않았다. 부동산 PF대출의 연체율은 전업권이 0.49%로 2016년 2.4%에 비해 대폭 하락했다. 고정이하 여신 잔액도 6 472억원으로 2016년말 1조 9 595억원 대비 큰 폭으로 감소했다. 김한정 의원은 부동산시장의 활황세를 등에 업고 부동산 PF대출이 급속히 증가하고 있다. 부동산 PF대출의 연체율이나 부실여신 잔액이 줄어드는 등 외관상으로는 문제가 없는 것으로 보이지만 부동산 PF대출의 잠재적 위험을 사전에 예방하는 것이 중요하다 고 강조했다. 이어 부동산 경기활황시 PF대출을 늘렸다 부동산 경기침체로 파산한 부산저축은행 사태도 있었다 며 감독당국은 부동산 PF대출에 대해 보다 적극적인 위험관리대책을 마련하여 부동산시장과 

In [9]:
text_clearing(test_doc)

'김한정 의원 4년만에 882% 증가감독당국 선제적 대응해야 자료 금융감독원 [쿠키뉴스] 유수환 기자 2일 국회 정무위원회 소속 더불어민주당 김한정 의원은 부동산 PF대출이 빠른 속도로 증가하고 있어 금융 불안을 초래할 위험이 커지고 있다며 감독당국에 보다 적극적인 위험 관리대책을 주문했다 금융감독원에 따르면 지난해 말 국내 금융권의 부동산 PF대출 잔액은 88조4838억원으로 2016년말 (47조256억원)에 대비 4년만에 41조4582억원(882%) 증가했다 부동산 PF대출 급증은 부동산경기 호조가 주된 배경이지만 코로나 영향으로 해외 대체투자가 어려운점(보험사) 부동산 직접투자 제한 영향(여전사 보험사) 등도 증가 요인이다 금융권역별로는 보험사가 36조3826억원으로 가장 많고 은행이 23조8572억원 저축은행 6조 8647억원 증권사 4조2691억원 상호금융 3조3105억원 순이다 부동산 PF대출 가운데 고정금리 비중은 792%였는데 저축은행이 976%로 가장 높았고 보험사 799% 여전사 723% 상호금융 610% 순이었다 부동산 PF대출의 금리는 보험사가 평균 324%로 가장 낮았고 상호금융 347% 여전사 459% 증권사 578% 저축은행 691% 순이었다 다만 아직 리스크에 노출되지는 않았다 부동산 PF대출의 연체율은 전업권이 049%로 2016년 24%에 비해 대폭 하락했다 고정이하 여신 잔액도 6472억원으로 2016년말 1조 9595억원 대비 큰 폭으로 감소했다 김한정 의원은 부동산시장의 활황세를 등에 업고 부동산 PF대출이 급속히 증가하고 있다 부동산 PF대출의 연체율이나 부실여신 잔액이 줄어드는 등 외관상으로는 문제가 없는 것으로 보이지만 부동산 PF대출의 잠재적 위험을 사전에 예방하는 것이 중요하다고 강조했다 이어 부동산 경기활황시 PF대출을 늘렸다 부동산 경기침체로 파산한 부산저축은행 사태도 있었다며 감독당국은 부동산 PF대출에 대해 보다 적극적인 위험관리대책을 마련하여 부동산시장과 금융시장의 안정을 도모해야 할 것이라고 촉구했다 s

In [10]:
clear_bracket_regex(test_doc)

'김한정 의원 "4년만에 88.2% 증가..감독당국 선제적 대응해야" \n\n\n\n\n\n            자료=금융감독원\n           \n\n\n             유수환 기자 = 2일 국회 정무위원회 소속 더불어민주당 김한정 의원은 부동산 PF대출이 빠른 속도로 증가하고 있어 금융 불안을 초래할 위험이 커지고 있다며 감독당국에 보다 적극적인 위험 관리대책을 주문했다.\n          \n금융감독원에 따르면 지난해 말 국내 금융권의 부동산 PF대출 잔액은 88조4,838억원으로 2016년말 에 대비 4년만에 41조4582억원 증가했다. 부동산 PF대출 급증은 부동산경기 호조가 주된 배경이지만 코로나 영향으로 해외 대체투자가 어려운점, 부동산 직접투자 제한 영향 등도 증가 요인이다.\n금융권역별로는 보험사가 36조3826억원으로 가장 많고 은행이 23조8572억원, 저축은행 6조 8647억원, 증권사 4조2691억원, 상호금융 3조3105억원 순이다.\n부동산 PF대출 가운데 고정금리 비중은 79.2%였는데, 저축은행이 97.6%로 가장 높았고, 보험사 79.9%, 여전사 72.3%, 상호금융 61.0% 순이었다. 부동산 PF대출의 금리는 보험사가 평균 3.24%로 가장 낮았고, 상호금융 3.47%, 여전사 4.59%, 증권사 5.78%, 저축은행 6.91% 순이었다.\n다만 아직 리스크에 노출되지는 않았다. 부동산 PF대출의 연체율은 전업권이 0.49%로 2016년 2.4%에 비해 대폭 하락했다. 고정이하 여신 잔액도 6,472억원으로 2016년말 1조 9,595억원 대비 큰 폭으로 감소했다.\n김한정 의원은 “부동산시장의 활황세를 등에 업고 부동산 PF대출이 급속히 증가하고 있다. 부동산 PF대출의 연체율이나 부실여신 잔액이 줄어드는 등 외관상으로는 문제가 없는 것으로 보이지만 부동산 PF대출의 잠재적 위험을 사전에 예방하는 것이 중요하다”고 강조했다.\n이어 “부동산 경기활황시 PF대출을 늘렸다 부동산 경기침체로 파산한 부산저축은행 

In [16]:
xplit(test_doc,['. ', '\n', '.\n'])

['김한정 의원 "4년만에 88.2% 증가..감독당국 선제적 대응해야" ',
 '',
 '',
 '',
 '',
 '',
 '            자료=금융감독원',
 '           ',
 '',
 '',
 '            [쿠키뉴스] 유수환 기자 = 2일 국회 정무위원회 소속 더불어민주당 김한정 의원은 부동산 PF대출이 빠른 속도로 증가하고 있어 금융 불안을 초래할 위험이 커지고 있다며 감독당국에 보다 적극적인 위험 관리대책을 주문했다',
 '          ',
 '금융감독원에 따르면 지난해 말 국내 금융권의 부동산 PF대출 잔액은 88조4,838억원으로 2016년말 (47조256억원)에 대비 4년만에 41조4582억원(88.2%) 증가했다',
 '부동산 PF대출 급증은 부동산경기 호조가 주된 배경이지만 코로나 영향으로 해외 대체투자가 어려운점(보험사), 부동산 직접투자 제한 영향(여전사, 보험사) 등도 증가 요인이다',
 '금융권역별로는 보험사가 36조3826억원으로 가장 많고 은행이 23조8572억원, 저축은행 6조 8647억원, 증권사 4조2691억원, 상호금융 3조3105억원 순이다',
 '부동산 PF대출 가운데 고정금리 비중은 79.2%였는데, 저축은행이 97.6%로 가장 높았고, 보험사 79.9%, 여전사 72.3%, 상호금융 61.0% 순이었다',
 '부동산 PF대출의 금리는 보험사가 평균 3.24%로 가장 낮았고, 상호금융 3.47%, 여전사 4.59%, 증권사 5.78%, 저축은행 6.91% 순이었다',
 '다만 아직 리스크에 노출되지는 않았다',
 '부동산 PF대출의 연체율은 전업권이 0.49%로 2016년 2.4%에 비해 대폭 하락했다',
 '고정이하 여신 잔액도 6,472억원으로 2016년말 1조 9,595억원 대비 큰 폭으로 감소했다',
 '김한정 의원은 “부동산시장의 활황세를 등에 업고 부동산 PF대출이 급속히 증가하고 있다',
 '부동산 PF대출의 연체율이나 부실여신 잔액이 줄어드는 등 외관상으로는 문제가 없는 것

In [49]:
def sentence_prep(text):
    cleared_sentences = []
    cleared = xplit(clear_bracket_regex(text), ['. ', '\n', '.\n', '\r\n', '.\r\n'])
    
    for sentence in cleared :
        if sentence and not junk_filter.match(sentence) :
            cleared_sentences.append(re.sub(r"[.]$", "", text_clearing(sentence)))
    return cleared_sentences

In [50]:
sents = sentence_prep(test_doc)

In [51]:
sents

['김한정 의원 4년만에 88.2% 증가 감독당국 선제적 대응해야',
 '금융감독원에 따르면 지난해 말 국내 금융권의 부동산 PF대출 잔액은 88조4838억원으로 2016년말 에 대비 4년만에 41조4582억원 증가했다',
 '부동산 PF대출 급증은 부동산경기 호조가 주된 배경이지만 코로나 영향으로 해외 대체투자가 어려운점 부동산 직접투자 제한 영향 등도 증가 요인이다',
 '금융권역별로는 보험사가 36조3826억원으로 가장 많고 은행이 23조8572억원 저축은행 6조 8647억원 증권사 4조2691억원 상호금융 3조3105억원 순이다',
 '부동산 PF대출 가운데 고정금리 비중은 79.2%였는데 저축은행이 97.6%로 가장 높았고 보험사 79.9% 여전사 72.3% 상호금융 61.0% 순이었다',
 '부동산 PF대출의 금리는 보험사가 평균 3.24%로 가장 낮았고 상호금융 3.47% 여전사 4.59% 증권사 5.78% 저축은행 6.91% 순이었다',
 '다만 아직 리스크에 노출되지는 않았다',
 '부동산 PF대출의 연체율은 전업권이 0.49%로 2016년 2.4%에 비해 대폭 하락했다',
 '고정이하 여신 잔액도 6472억원으로 2016년말 1조 9595억원 대비 큰 폭으로 감소했다',
 '김한정 의원은 부동산시장의 활황세를 등에 업고 부동산 PF대출이 급속히 증가하고 있다',
 '부동산 PF대출의 연체율이나 부실여신 잔액이 줄어드는 등 외관상으로는 문제가 없는 것으로 보이지만 부동산 PF대출의 잠재적 위험을 사전에 예방하는 것이 중요하다고 강조했다',
 '이어 부동산 경기활황시 PF대출을 늘렸다 부동산 경기침체로 파산한 부산저축은행 사태도 있었다며 감독당국은 부동산 PF대출에 대해 보다 적극적인 위험관리대책을 마련하여 부동산시장과 금융시장의 안정을 도모해야 할 것이라고 촉구했다',
 '갓 구워낸 바삭바삭한 뉴스 ⓒ 쿠키뉴스 무단전재 및 재배포금지']

## 2. 토크나이징

각 문장들을 의미 단위(형태소)로 쪼개는 과정

토큰을 활용하여 문서를 BOW(Bag of Words)로 표현하기

In [28]:
from konlpy.tag import Okt, Kkma, Komoran

okt = Okt() # tag가 가장 작음
kkma = Kkma()
komo = Komoran()

In [52]:
print(okt.morphs(sents[1]))

['금융감독원', '에', '따르면', '지난해', '말', '국내', '금융', '권', '의', '부동산', 'PF', '대출', '잔액', '은', '88조', '4838억원', '으로', '2016년', '말', '에', '대비', '4년', '만에', '41조', '4582억원', '증가', '했다']


In [53]:
print(kkma.morphs(sents[1]))

['금융', '감독원', '에', '따르', '면', '지난해', '말', '국내', '금융권', '의', '부동산', 'PF', '대출', '잔액', '은', '88', '조', '4838', '억', '원', '으로', '2016', '년', '말', '에', '대비', '4', '년', '만', '에', '41', '조', '4582', '억', '원', '증가', '하', '었', '다']


In [34]:
print(okt.nouns(sents[1]))

['금융감독원', '지난해', '말', '국내', '금융', '부동산', '대출', '잔액', '대비', '증가']


In [37]:
print(okt.pos(sents[1]))

[('금융감독원', 'Noun'), ('에', 'Josa'), ('따르면', 'Verb'), ('지난해', 'Noun'), ('말', 'Noun'), ('국내', 'Noun'), ('금융', 'Noun'), ('권', 'Suffix'), ('의', 'Josa'), ('부동산', 'Noun'), ('PF', 'Alpha'), ('대출', 'Noun'), ('잔액', 'Noun'), ('은', 'Josa'), ('88조', 'Number'), ('4838억원', 'Number'), ('으로', 'Foreign'), ('2016년', 'Number'), ('말', 'Foreign'), ('에', 'Josa'), ('대비', 'Noun'), ('4년', 'Number'), ('만에', 'Foreign'), ('41조', 'Number'), ('4582억원', 'Number'), ('증가', 'Noun'), ('했다', 'Verb')]


In [39]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [54]:
from konlpy.tag import Okt

pos_set = ['Noun', 'Verb', 'Alpha']

def selected_tokenizer(sent, pos = pos_set) :
    t = Okt()
    return [x[0] for x in t.pos(sent) if len(x[0]) > 1 and x[1] in pos]

print(selected_tokenizer(sents[1]))

['금융감독원', '따르면', '지난해', '국내', '금융', '부동산', 'PF', '대출', '잔액', '대비', '증가', '했다']


In [43]:
tokens = [selected_tokenizer(x) for x in sents]
print(tokens)

[['김한정', '의원', '증가', '감독', '당국', '선제', '대응', '해야'], ['금융감독원', '따르면', '지난해', '국내', '금융', '부동산', 'PF', '대출', '잔액', '대비', '증가', '했다'], ['부동산', 'PF', '대출', '급증', '부동산', '경기', '호조', '배경', '코로나', '영향', '해외', '대체', '투자가', '부동산', '직접투자', '제한', '영향', '증가', '요인'], ['금융', '별로', '보험사', '가장', '은행', '저축은행', '증권사', '상호금융', '순이'], ['부동산', 'PF', '대출', '가운데', '고정', '금리', '비중', '였는데', '저축은행', '가장', '보험사', '전사', '상호금융', '이었다'], ['부동산', 'PF', '대출', '금리', '보험사', '평균', '가장', '상호금융', '전사', '증권사', '저축은행', '이었다'], ['다만', '리스크', '노출', '되지는', '않았다'], ['부동산', 'PF', '대출', '연체', '전업', '비해', '대폭', '하락', '했다'], ['고정', '이하', '여신', '잔액', '대비', '감소', '했다'], ['김한정', '의원', '부동산', '시장', '활황', '업고', '부동산', 'PF', '대출', '증가'], ['부동산', 'PF', '대출', '연체', '부실', '여신', '잔액', '줄어드는', '외관', '문제', '보이', '부동산', 'PF', '대출', '잠재', '위험', '사전', '예방', '하는', '강조', '했다'], ['이어', '부동산', '경기', '활황', 'PF', '대출', '늘렸다', '부동산', '침체', '부산', '저축은행', '사태', '감독', '당국', '부동산', 'PF', '대출', '대해', '보다', '적극', '위험관리', '대책', '마련', '하여', '부동산', '시장', '금융시장',

In [94]:
from collections import Counter

token_all = [x for token in tokens for x in token] #list of list -> list
c = Counter()
c.update(token_all) #Counter에 list 추가 

In [95]:
#BOW <- Term Frequency
c.most_common(10)

[('부동산', 15),
 ('PF', 10),
 ('대출', 10),
 ('했다', 5),
 ('증가', 4),
 ('저축은행', 4),
 ('잔액', 3),
 ('보험사', 3),
 ('가장', 3),
 ('상호금융', 3)]

## 3. Vectorization

문서 set을 모두 bow로 만든 다음 matrix로 문서 set을 표현
* 행(row)은 각 문서를 의미 -> 문서의 벡터화
* 열(column)은 각 단어들로 이루어진 차원
* TfIdf(Term frequency - Inverse document frequency)
  * idf 식 설명 : https://wikidocs.net/31698

In [116]:
from sklearn.feature_extraction import DictVectorizer # 딕셔너리로 넣으면 벡터화
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  # 리스트 of 리스트로 넣으면 벡터화

def bow2vec(bow_list) :
    '''
    input : a list of bow lists
    output : vectors
    '''
    data_bow = []
    for item in bow_list :
        c = Counter()
        c.update(item)
        data_bow.append(c)
    vectorizer = DictVectorizer()
    X = vectorizer.fit_transform(data_bow)
    features = vectorizer.get_feature_names()

    return X, features

def bow2tfidfvec(bow_list) :
    '''
    input : a list of bow lists
    output : vectors
    '''
    data_bow = []
    for item in bow_list :
        data_bow.append(" ".join(item))
    tfidfvectorizer = TfidfVectorizer()
    X = tfidfvectorizer.fit_transform(data_bow)
    features = tfidfvectorizer.get_feature_names()

    return X, features

In [118]:
tfvec, features_tf = bow2vec(tokens)
tfidfvec, features_tfidf = bow2tfidfvec(tokens)

In [119]:
print(len(tokens), ',' ,len(c))
print(tfvec.shape)

13 , 96
(13, 96)


In [120]:
print(tfvec[0])
print(tfidfvec[0])

  (0, 3)	1.0
  (0, 16)	1.0
  (0, 21)	1.0
  (0, 23)	1.0
  (0, 50)	1.0
  (0, 66)	1.0
  (0, 78)	1.0
  (0, 91)	1.0
  (0, 91)	0.3465285489529673
  (0, 23)	0.40183588026661626
  (0, 50)	0.40183588026661626
  (0, 21)	0.3465285489529673
  (0, 3)	0.3465285489529673
  (0, 78)	0.2768495520376762
  (0, 66)	0.3465285489529673
  (0, 16)	0.3465285489529673


In [125]:
#DictVectorizer, TfidfVectorizer 둘의 feature 차이 비교
#영문 표현 방식 외 단어 idx 순서는 같음
for a, b in zip(features_tf, features_tfidf) :
    if a != b :
        print(a, b)

PF pf


In [161]:
#features 단어 일부 확인
features_tfidf[:10]

['pf', '가운데', '가장', '감독', '감소', '강조', '경기', '고정', '구워', '국내']

In [157]:
#Vectorizer 학습된 Matrix는 Scipy의 csr_matrix 형태로 저장되어 있음
tfidfvec[0]

<1x96 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [146]:
#0번 문서의 단어 리스트 및 vector값 확인
print(tfidfvec[0].indices)
print(tfidfvec[0].data)

[91 23 50 21  3 78 66 16]
[0.34652855 0.40183588 0.40183588 0.34652855 0.34652855 0.27684955
 0.34652855 0.34652855]


In [147]:
#sparse matrix -> 일반(dense) matrix로 변환
tfidfvec[0].toarray()

array([[0.        , 0.        , 0.        , 0.34652855, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.34652855, 0.        , 0.        , 0.        ,
        0.        , 0.34652855, 0.        , 0.40183588, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.40183588, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.34652855, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [162]:
#(단어 idx, 단어, tfidf) 값으로 문서를 저장
words = {}
for e, item in enumerate(tfidfvec) :
    words[e] = []
    for idx, val in zip(item.indices, item.data) :
        words[e].append((idx, features_tfidf[idx] ,val))
words[0]

[(91, '해야', 0.3465285489529673),
 (23, '대응', 0.40183588026661626),
 (50, '선제', 0.40183588026661626),
 (21, '당국', 0.3465285489529673),
 (3, '감독', 0.3465285489529673),
 (78, '증가', 0.2768495520376762),
 (66, '의원', 0.3465285489529673),
 (16, '김한정', 0.3465285489529673)]

In [154]:
#문서에서 tfidf 높은 단어 순으로 출력
for idx, keys in words.items() :
    keys = sorted(keys, key = lambda x : -x[2])
    print(idx, '-', keys[:2])

0 - [(23, '대응', 0.40183588026661626), (50, '선제', 0.40183588026661626)]
1 - [(9, '국내', 0.3613465483633835), (80, '지난해', 0.3613465483633835)]
2 - [(59, '영향', 0.47333657195458456), (42, '부동산', 0.3475015213113972)]
3 - [(51, '순이', 0.3858632539920651), (65, '은행', 0.3858632539920651)]
4 - [(58, '였는데', 0.3347810487425642), (45, '비중', 0.3347810487425642)]
5 - [(87, '평균', 0.3800789860553179), (68, '이었다', 0.3277662000662475)]
6 - [(54, '않았다', 0.447213595499958), (30, '되지는', 0.447213595499958)]
7 - [(89, '하락', 0.413253801615047), (27, '대폭', 0.413253801615047)]
8 - [(4, '감소', 0.4381495653859565), (69, '이하', 0.4381495653859565)]
9 - [(55, '업고', 0.4121606488111609), (42, '부동산', 0.40345203525442114)]
10 - [(5, '강조', 0.24738607361225384), (88, '하는', 0.24738607361225384)]
11 - [(42, '부동산', 0.3762172900466852), (82, '촉구', 0.19216901739240796)]
12 - [(14, '금지', 0.408248290463863), (37, '배포', 0.408248290463863)]
